## Data Intensive Distributed Computing

In [220]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/sql-data.tgz
!tar -xzf sql-data.tgz

Next, we launch Spark.  When I used the RDD interface for previous assignments, I created a `SparkContext` when I launched Spark.   To use Spark SQL and the DataFrame interface, I instead create a `SparkSession`.   I do that as shown in the next cell (run it!).    

In [221]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
import random

spark = SparkSession.builder.appName("IrTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

Next, let's create DataFrames from the TPC-H data files.  We have already downloaded the TPC-H data files.

There is one file for each table in the TPC-H database, e.g., `nation.tbl` for the TPC-H Nation table, `customer.tbl` for the TPC-H Customer table, and so on.    These are plain text csv files, with the character `|` used as a field separator.

Create a Spark DataFrame corresponding to the TPC-H Nation table by loading the data from the `nation.tbl` file.   Run the code in the next cell to do this.   After I run this code, `nation_raw` will refer to Ir new Spark DataFrame.   The Spark `show()` method will display a small (default 20) number of elements from the DataFrame, so that I can inspect them. 

In [222]:
nation_raw = spark.read.csv("nation.tbl",sep='|',inferSchema=True)
nation_raw.show()

+---+----------+---+--------------------+----+
|_c0|       _c1|_c2|                 _c3| _c4|
+---+----------+---+--------------------+----+
|  0|   ALGERIA|  0| haggle. carefull...|null|
|  1| ARGENTINA|  1|al foxes promise ...|null|
|  2|    BRAZIL|  1|y alongside of th...|null|
|  3|    CANADA|  1|eas hang ironic, ...|null|
|  4|     EGYPT|  4|y above the caref...|null|
|  5|  ETHIOPIA|  0|ven packages wake...|null|
|  6|    FRANCE|  3|refully final req...|null|
|  7|   GERMANY|  3|l platelets. regu...|null|
|  8|     INDIA|  2|ss excuses cajole...|null|
|  9| INDONESIA|  2| slyly express as...|null|
| 10|      IRAN|  4|efully alongside ...|null|
| 11|      IRAQ|  4|nic deposits boos...|null|
| 12|     JAPAN|  2|ously. final, exp...|null|
| 13|    JORDAN|  4|ic deposits are b...|null|
| 14|     KENYA|  0| pending excuses ...|null|
| 15|   MOROCCO|  0|rns. blithely bol...|null|
| 16|MOZAMBIQUE|  0|s. ironic, unusua...|null|
| 17|      PERU|  1|platelets. blithe...|null|
| 18|     CHI

Now I have a DataFrame to work with.   The columns of the DataFrame correspond to the fields of the TPC-H Nation table, so have a look at the TPC-H schema diagram to see what I are dealing with.   Column c0 is the NATIONKEY, column c1 is the NAME, c2 is the REGIONKEY, and so on.   Since this is a synthetic database, I'll notice that the data in some of the fields (like the COMMENT field) consists of random words.   That's fine.   I can also ask Spark to tell I about the type of data in each column:

In [223]:
nation_raw.dtypes

[('_c0', 'int'),
 ('_c1', 'string'),
 ('_c2', 'int'),
 ('_c3', 'string'),
 ('_c4', 'string')]

Before going on, let's clean this DataFrame up a bit, to make it easier to use.   First, let's assign names to the columns, so that we can remember what information each column holds.   Second, I'll notice that Spark has created an extra final column (filled with `null` values) because each line in the input file ends with a separator character (|).  Let's drop that final column, since we don't need it.   Run the following code to do this:

In [224]:
nation = nation_raw.toDF('NationKey','Name','RegionKey','Comment','extra').drop('extra').cache()
nation.show()

+---------+----------+---------+--------------------+
|NationKey|      Name|RegionKey|             Comment|
+---------+----------+---------+--------------------+
|        0|   ALGERIA|        0| haggle. carefull...|
|        1| ARGENTINA|        1|al foxes promise ...|
|        2|    BRAZIL|        1|y alongside of th...|
|        3|    CANADA|        1|eas hang ironic, ...|
|        4|     EGYPT|        4|y above the caref...|
|        5|  ETHIOPIA|        0|ven packages wake...|
|        6|    FRANCE|        3|refully final req...|
|        7|   GERMANY|        3|l platelets. regu...|
|        8|     INDIA|        2|ss excuses cajole...|
|        9| INDONESIA|        2| slyly express as...|
|       10|      IRAN|        4|efully alongside ...|
|       11|      IRAQ|        4|nic deposits boos...|
|       12|     JAPAN|        2|ously. final, exp...|
|       13|    JORDAN|        4|ic deposits are b...|
|       14|     KENYA|        0| pending excuses ...|
|       15|   MOROCCO|      

This style of code should look familar to I.  We started with the `nation_raw` DataFrame and applied a series of DataFrame operations (`toDF`, `drop`, and `cache`).   This is just like the RDD interface, except now we are applying DataFrame operations to DataFrames, instead of RDD operations to RDDs.

Next, let's load up the TPC-H Supplier table, and then try performing some queries:

In [225]:
supplier_raw = spark.read.csv("supplier.tbl",sep='|',inferSchema=True).drop("_c7")
supplier = supplier_raw.toDF("SuppKey","Name","Address","NationKey","Phone","AcctBal","Comment").cache()
supplier.show()

+-------+------------------+--------------------+---------+---------------+-------+--------------------+
|SuppKey|              Name|             Address|NationKey|          Phone|AcctBal|             Comment|
+-------+------------------+--------------------+---------+---------------+-------+--------------------+
|      1|Supplier#000000001| N kD4on9OM Ipw3,...|       17|27-918-335-1736|5755.94|each slyly above ...|
|      2|Supplier#000000002|89eJ5ksX3ImxJQBvx...|        5|15-679-861-2259|4032.68| slyly bold instr...|
|      3|Supplier#000000003|q1,G3Pj6OjIuUYfUo...|        1|11-383-516-1199| 4192.4|blithely silent r...|
|      4|Supplier#000000004|Bk7ah4CK8SYQTepEm...|       15|25-843-787-7479|4641.08|riously even requ...|
|      5|Supplier#000000005|   Gcdm2rJRzl5qlTVzc|       11|21-151-690-3663|-283.84|. slyly regular p...|
|      6|Supplier#000000006|        tQxuVm7s7CnK|       14|24-696-997-4969|1365.79|final accounts. r...|
|      7|Supplier#000000007|s,4TicNGB4uO6PaSq...|      

---
#### Writing Queries
There are two equivalent ways of writing queries over Spark DataFrames.   The first way is to assign a "view name" to the DataFrame, and then write SQL queries referring to those view names using the `sql` operation.  

The code below gives the view names "nation" and "supplier" to the two DataFrames we've already created.

In [226]:
supplier.createOrReplaceTempView("supplier")
nation.createOrReplaceTempView("nation")

Now, we can write SQL queries that refer to the "supplier" and "nation" views as tables.   For example, suppose we want to see the names and addresses of suppliers who have account balances above 9900.00:

In [227]:
q1_result = spark.sql("select Name, Address, AcctBal from supplier where AcctBal > 9900.00")
q1_result.show()

+------------------+--------------------+-------+
|              Name|             Address|AcctBal|
+------------------+--------------------+-------+
|Supplier#000000049|     Nvq 6macF4GtJvz|9915.24|
|Supplier#000000234|iMrk7HUD87at3IIh4rBi| 9957.0|
|Supplier#000000693|S,mnHfsroFOVieQGd...|9956.55|
|Supplier#000000855|ekQwhb9fh5VGIvMBJ...|9964.88|
|Supplier#000000892|j6prA4M3sX9a9xHem...|9993.46|
+------------------+--------------------+-------+



In the example above, the `sql` command runs the SQL query against the supplier table.   It returns the query result as a new DataFrame, which `q1_result` refers to.

Instead of writing Ir queries in SQL and running them using `sql`, it is possible to do the same thing by applying a sequence of DataFrame operations to the input DataFrames, as I did when I were using the RDD interface in the previous assignments.    For example, to answer the same query that we just answered using SQL, we can do the following:

In [228]:
q1_resultB = supplier.filter("AcctBal > 9900.00").select('Name','Address','AcctBal')
q1_resultB.show()

+------------------+--------------------+-------+
|              Name|             Address|AcctBal|
+------------------+--------------------+-------+
|Supplier#000000049|     Nvq 6macF4GtJvz|9915.24|
|Supplier#000000234|iMrk7HUD87at3IIh4rBi| 9957.0|
|Supplier#000000693|S,mnHfsroFOVieQGd...|9956.55|
|Supplier#000000855|ekQwhb9fh5VGIvMBJ...|9964.88|
|Supplier#000000892|j6prA4M3sX9a9xHem...|9993.46|
+------------------+--------------------+-------+



In [229]:
def load_dataset_and_set_views():
    global supplier, orders, customer, partsupp, nation, part
    
    supplier_raw = spark.read.csv("supplier.tbl",sep='|',inferSchema=True).drop("_c7")
    supplier = supplier_raw.toDF("SuppKey","Name","Address","NationKey","Phone","AcctBal","Comment").cache()
    supplier.createOrReplaceTempView("supplier")
    
    # Ir solution to Question 0 here
    order_raw = spark.read.csv("orders.tbl",sep='|',inferSchema=True).drop("_c9")
    orders = order_raw.toDF("OrderKey", "CustKey", "OrderStatus", "TotalPrice", "OrderDate", "OrderPriority", "Clerk", "ShipPriority", "Comment").cache()
    orders.createOrReplaceTempView("orders")

    customer_raw = spark.read.csv("customer.tbl",sep='|',inferSchema=True).drop("_c8")
    customer = customer_raw.toDF("CustKey","Name", "Address", "NationKey", "Phone","AccBal","MktSegment", "Comment").cache()
    customer.createOrReplaceTempView("customer")
    
    partsupp_raw = spark.read.csv("partsupp.tbl",sep='|',inferSchema=True).drop("_c5")
    partsupp = partsupp_raw.toDF("PartKey", "SuppKey", "AvailQty", "SupplyCost", "Comment").cache()
    partsupp.createOrReplaceTempView("partsupp")
    
    nation_raw = spark.read.csv("nation.tbl",sep='|',inferSchema=True).drop("_c4")
    nation = nation_raw.toDF("NationKey", "Name", "RegionKey", "Comment").cache()
    nation.createOrReplaceTempView("nation")
    
    part_raw = spark.read.csv("part.tbl",sep='|',inferSchema=True).drop("_c9")
    part = part_raw.toDF("PartKey", "Name", "Mfgr", "Brand", "Type", "Size", "Container", "RetailPrice", "Comment").cache()
    part.createOrReplaceTempView("part")

In [230]:
# Ir tests here
load_dataset_and_set_views()

In [231]:
def five_highest_totalprice_orders_sql():
    # Ir solution to Question 1 here
    query = "select OrderKey, OrderDate, TotalPrice from orders order by TotalPrice desc limit 5"
    return spark.sql(query)


In [232]:
# Ir tests here
five_highest_totalprice_orders_sql().show()

+--------+-------------------+----------+
|OrderKey|          OrderDate|TotalPrice|
+--------+-------------------+----------+
|  279812|1994-02-19 00:00:00| 479129.21|
|  370726|1996-09-29 00:00:00|  460099.4|
|   66659|1993-10-15 00:00:00| 458396.42|
|  253639|1998-01-23 00:00:00| 456532.89|
|  502886|1994-04-12 00:00:00| 456423.88|
+--------+-------------------+----------+



In [234]:
def five_highest_totalprice_orders_dtf():
    # Ir solution to Question 2 here
    from pyspark.sql.functions import desc
    return orders.orderBy(desc('TotalPrice')).select('OrderKey', 'OrderDate', 'TotalPrice').limit(5)

In [235]:
# Ir tests here
five_highest_totalprice_orders_dtf().show(5)

+--------+-------------------+----------+
|OrderKey|          OrderDate|TotalPrice|
+--------+-------------------+----------+
|  279812|1994-02-19 00:00:00| 479129.21|
|  370726|1996-09-29 00:00:00|  460099.4|
|   66659|1993-10-15 00:00:00| 458396.42|
|  253639|1998-01-23 00:00:00| 456532.89|
|  502886|1994-04-12 00:00:00| 456423.88|
+--------+-------------------+----------+



In [236]:
def cust_most_recent_order_sql(custkey):
    # Ir solution to Question 3 here
    query = "select c.Name, o.OrderDate, o.TotalPrice from customer as c inner join orders as o on o.CustKey = c.CustKey where c.CustKey = {} order by OrderDate desc limit 1".format(custkey)
    return spark.sql(query)

In [237]:
# Ir tests here
cust_most_recent_order_sql(1).show()

+------------------+-------------------+----------+
|              Name|          OrderDate|TotalPrice|
+------------------+-------------------+----------+
|Customer#000000001|1997-03-04 00:00:00| 268835.44|
+------------------+-------------------+----------+



In [238]:
def cust_most_recent_order_dtf(custkey):
    # Ir solution to Question 4 here
    from pyspark.sql.functions import desc
    out_df = customer.filter('CustKey == {}'.format(custkey)).join(orders, orders.CustKey == customer.CustKey, "inner").orderBy(desc('OrderDate')).select("Name", "OrderDate", "TotalPrice")

    return out_df.limit(1)

In [239]:
# Ir tests here
cust_most_recent_order_dtf(1).show(10)

+------------------+-------------------+----------+
|              Name|          OrderDate|TotalPrice|
+------------------+-------------------+----------+
|Customer#000000001|1997-03-04 00:00:00| 268835.44|
+------------------+-------------------+----------+



In [240]:
def distinct_supplied_parts(nname):
    # Ir solution to Question 5 here
    query = "select count(distinct(ps.PartKey)) from partsupp as ps inner join supplier as s on ps.SuppKey = s.SuppKey inner join nation as n on n.NationKey == s.NationKey where n.Name = '{}'".format(nname)
    #query = "select n.NationKey, n.Name from supplier as s inner join nation as n on n.NationKey = s.NationKey"
    return spark.sql(query).collect()[0][0]

In [241]:
# Ir tests here
distinct_supplied_parts("CANADA")

2799

In [301]:
def count_suppliers_brand_per_nation(bname):
    # Ir solution to Question 6 here
    subquery = "select distinct(ps.SuppKey) from part as p inner join partsupp as ps on ps.PartKey = p.PartKey where p.Brand = '{}'".format(bname)
    outer_query = "select n.Name, count(distinct(s.SuppKey)) as Suppliers from supplier as s inner join nation as n on s.NationKey = n.NationKey where s.SuppKey in ({}) group by n.Name order by n.Name".format(subquery)
    return spark.sql(outer_query)

In [302]:
# Ir tests here
count_suppliers_brand_per_nation("Brand#14").show()

+----------+---------+
|      Name|Suppliers|
+----------+---------+
|   ALGERIA|       34|
| ARGENTINA|       38|
|    BRAZIL|       41|
|    CANADA|       35|
|     CHINA|       51|
|     EGYPT|       39|
|  ETHIOPIA|       32|
|    FRANCE|       35|
|   GERMANY|       49|
|     INDIA|       45|
| INDONESIA|       45|
|      IRAN|       39|
|      IRAQ|       40|
|     JAPAN|       40|
|    JORDAN|       28|
|     KENYA|       35|
|   MOROCCO|       40|
|MOZAMBIQUE|       32|
|      PERU|       37|
|   ROMANIA|       32|
+----------+---------+
only showing top 20 rows



In [299]:
def order_number_per_customer_nation(nname):
    # Ir solution to Question 7 here
    query = "select year(o.OrderDate) as Year, count(o.OrderKey) as Orders from nation as n inner join customer as c on c.NationKey = n.NationKey inner join orders as o on o.CustKey = c.CustKey where n.Name = '{}' group by year(o.OrderDate) order by count(o.OrderKey) desc".format(nname)
    
    return spark.sql(query)


In [300]:
# Ir tests here
order_number_per_customer_nation("CANADA").show()

+----+------+
|Year|Orders|
+----+------+
|1992|   982|
|1996|   940|
|1995|   932|
|1997|   921|
|1994|   912|
|1993|   900|
|1998|   595|
+----+------+

